# Bibliotheksstatistik

Die [Deutsche Bibliotheksstatistik](https://www.bibliotheksstatistik.de/) kann frei abgerufen werden.

Für die Auswertung habe ich den Zeitraum von **2010 bis 2020** gewählt.

Folgende Variablen habe ich ausgewählt:
- Ausgaben Erwerbungen
- Bestand (virtuell)
- Bestand (insgesamt)
- Entleihende (ab 60 J.)
- Entleihende (bis 12 J.)
- Entleihende (insgesamt)
- Entleihen (virtueller Bestand)
- Entleihen (gesamt)
- Laufende Ausgaben

Das Ergebnis habe ich als **csv-Datei** heruntergeladen.

## Daten bereinigen

In [53]:
# import modules
import pandas as pd

df = pd.read_csv('./data/dbs_data.csv', delimiter=';', header=[2,3], encoding='ISO-8859-1', low_memory=False)

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8555 entries, 0 to 8554
Columns: 104 entries, ('NR', 'Unnamed: 0_level_1') to ('Unnamed: 103_level_0', 'Unnamed: 103_level_1')
dtypes: float64(1), object(103)
memory usage: 6.8+ MB
None
                  NR               Name  \
  Unnamed: 0_level_1 Unnamed: 1_level_1   
0                  1            Aach GB   
1                  2          Aachen GB   
2                  3          Aachen GB   
3                  4          Aachen GB   
4                  5          Aachen GB   

                                          Bibliothek             DBS-ID  \
                                  Unnamed: 2_level_1 Unnamed: 3_level_1   
0            Kath. Öffentliche Bücherei St. Nikolaus              QN043   
1            Kath. Öffentliche Bibliothek St. Konrad              PY065   
2        Kath. Öffentliche Bibliothek St. Laurentius              QU719   
3           Kath. Öffentliche Bibliothek St. Severin              NH551   
4  Bücherins

Im Dataframe sind nun alle Daten aus den verschiedenen öffentlichen Bibliotheken in Deutschland gesammelt. Ich brauche jedoch nur die Summe, also die Daten für ganz Deutschland. Diese befinden sich in einer Zeile namens *Summe*. Deshalb übernehme ich nur diese.

In [54]:
sum_row = df.loc[(df[('NR','Unnamed: 0_level_1')] == 'Summe')]

print(sum_row)

                     NR               Name         Bibliothek  \
     Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
8547              Summe                NaN                NaN   

                 DBS-ID  Entleiher                                   \
     Unnamed: 3_level_1       2010       2011       2012       2013   
8547                NaN  7.844.530  7.725.936  7.604.293  7.479.921   

                            ... Ausg. Erwerbung                            \
           2014       2015  ...            2012         2013         2014   
8547  7.355.708  7.249.936  ...     100.690.622  104.213.606  104.353.361   

                                                                          \
             2015         2016         2017         2018            2019   
8547  104.934.760  108.944.525  109.927.695  111.316.147  114.191.003,07   

                     Unnamed: 103_level_0  
                2020 Unnamed: 103_level_1  
8547  111.839.663,17                  NaN

Die Zahlen haben Punkte als Tausendertrennzeichen. Diese müssen entfernt werden. Dafür wandle ich sie in Strings um, damit ich ```str.replace()``` anwenden kann. Als Dezimaltrennzeichen ersetze ich das Komma durch einen Punkt.

In [55]:
library_df = sum_row.astype('string')

for label, content in library_df.iteritems():
    library_df[label] = library_df[label].str.replace('.', '', regex=True)
    library_df[label] = library_df[label].str.replace(',', '.', regex=True)

library_df

NR               Name         Bibliothek  \
     Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
8547              Summe               <NA>               <NA>   

                 DBS-ID Entleiher                                      \
     Unnamed: 3_level_1      2010     2011     2012     2013     2014   
8547               <NA>   7844530  7725936  7604293  7479921  7355708   

               ... Ausg. Erwerbung                                   \
         2015  ...            2012       2013       2014       2015   
8547  7249936  ...       100690622  104213606  104353361  104934760   

                                                                   \
           2016       2017       2018          2019          2020   
8547  108944525  109927695  111316147  114191003.07  111839663.17   

     Unnamed: 103_level_0  
     Unnamed: 103_level_1  
8547                 <NA>  

[1 rows x 104 columns]

Die Spaltennamen sind immer in eine Variable wie beispielsweise *Ausgaben Erwerbung* und dann in die Jahre eingeteilt. Ich möchte jedoch, dass jedes Jahr eine Spalte ist und jede Variable eine Zeile. So ist es besser lesbar. Die ersten fünf und die letzte Zeile brauche ich gar nicht.

In [56]:
library_df = pd.melt(library_df)

library_df = library_df.drop([0, 1, 2, 3, 103])

library_df = library_df.pivot(index='variable_0', columns='variable_1', values='value')

print(library_df)

variable_1             2010       2011       2012       2013       2014  \
variable_0                                                                
Ausg. Erwerbung    96842615   99526558  100690622  104213606  104353361   
Best. virt.Best.     457360     591069     829951    1265499    1622216   
Bestand insges    123027730  122796523  122025276  121235283  119858476   
Entl. ab 60 J.       737336     760140     805601     819270     846045   
Entl. bis 12 J.     2076475    2081832    2034793    2015921    2003788   
Entl. virt.Best.     961683    1443091    3418398    7060821   11436854   
Entleih. insges.  375793896  378719659  375565974  373419034  362542582   
Entleiher           7844530    7725936    7604293    7479921    7355708   
Lfd. Ausgaben     822217216  845950958  858229280  874839980  902104963   

variable_1             2015       2016       2017       2018           2019  \
variable_0                                                                    
Ausg. Erwerbung 

Damit mit den Werten später gerechnet werden kann, müssen sie noch in Floats umgewandelt werden.

In [70]:
for label, content in library_df.iteritems():
    library_df[label] = library_df[label].astype(float)

library_df

variable_1,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
variable_0,,,,,,,,,,,
Ausg. Erwerbung,96842615.0,99526558.0,100690622.0,104213606.0,104353361.0,104934760.0,108944525.0,109927695.0,111316147.0,1.141910e+08,1.118397e+08
Best. virt.Best.,457360.0,591069.0,829951.0,1265499.0,1622216.0,1134853.0,1491330.0,1305260.0,1780716.0,2.977275e+06,4.440515e+06
Bestand insges,123027730.0,122796523.0,122025276.0,121235283.0,119858476.0,117511823.0,116080149.0,113808636.0,112176481.0,1.108133e+08,1.080853e+08
Entl. ab 60 J.,737336.0,760140.0,805601.0,819270.0,846045.0,868571.0,919600.0,932875.0,970320.0,9.999570e+05,9.448310e+05
Entl. bis 12 J.,2076475.0,2081832.0,2034793.0,2015921.0,2003788.0,1973670.0,2034986.0,2028492.0,2063797.0,2.108134e+06,1.706033e+06
Entl. virt.Best.,961683.0,1443091.0,3418398.0,7060821.0,11436854.0,16314414.0,19813365.0,24350279.0,28979729.0,3.341006e+07,4.143244e+07
Entleih. insges.,375793896.0,378719659.0,375565974.0,373419034.0,362542582.0,361763984.0,355303531.0,344544584.0,338358540.0,3.383572e+08,2.610734e+08
Entleiher,7844530.0,7725936.0,7604293.0,7479921.0,7355708.0,7249936.0,7431335.0,7306049.0,7264051.0,7.309666e+06,6.399536e+06
Lfd. Ausgaben,822217216.0,845950958.0,858229280.0,874839980.0,902104963.0,921825080.0,948461369.0,946273426.0,971207532.0,1.006269e+09,1.014799e+09


In [67]:
library_df.dtypes

variable_1
2010    float64
2011    float64
2012    float64
2013    float64
2014    float64
2015    float64
2016    float64
2017    float64
2018    float64
2019    float64
2020    float64
dtype: object